# HEP Data

This noteboook reads in the outputs of combine to prepare tables of limits of the different points in the parameter space.

In [2]:
import numpy as np
import glob
import os
import uproot
import json
from tabulate import tabulate
import plot_utils as plot

In [13]:
def create_limit_table(params):
    """
    Expects params to be the output for get_scan_limits.
    Returns table of limits.
    """
    rows = np.array(['mS', 'mPhi', 'T', 'Mode', 'Xsec', 'Obs', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma'])
    for (param, lims) in params:

        _exp =lims[1][2]
        _s1p =lims[1][1] 
        _s1m =lims[1][3] 
        _s2p =lims[1][0] 
        _s2m =lims[1][4] 
        _obs =lims[1][5]   

        newline = [*param, _obs, _exp, _s2p, _s1p, _s1m, _s2m]
        rows = np.row_stack((rows, newline))
    
    return rows

In [14]:
# script parameters
ofile = '../approval/hep_{}_{}.txt'
itag = '../approval/'
HT = '1000'

In [9]:
asymptotic_params = plot.get_scan_limits(path=itag, file='../config/xsections_SUEP.json', method='AsymptoticLimits')
asymptotic_table = create_limit_table(asymptotic_params)

Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T4p00_mS125.000_mPhi2.000_T4.000_modeleptonic_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T4p00_mS125.000_mPhi4.000_T4.000_modeleptonic_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T6p00_mS125.000_mPhi3.000_T6.000_modegeneric_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T6p00_mS125.000_mPhi3.000_T6.000_modeleptonic_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T8p00_mS125.000_mPhi2.000_T8.000_modeleptonic_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T8p00_mS125.000_mPhi2.000_T8.000_modehadronic_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root
Bad limits ../approval/higgsCombineGluGluToSUEP_HT1000_T8p00_mS125.000_mPhi4.000_T8.000_modelep

In [12]:
toys_params = plot.get_scan_limits(path=itag, file='../config/xsections_SUEP.json', method='HybridNew')
toys_table = create_limit_table(toys_params)

'path' must be a string, pathlib.Path, an object with 'read' and 'seek' methods, or a length-1 dict of {file_path: object_path}, not None
File doesn't exit ../approval/higgsCombineGluGluToSUEP_HT1000_T1p00_mS200.000_mPhi4.000_T1.000_modeleptonic_TuneCP5_13TeV-pythia8.HybridNew.mH125.root
'path' must be a string, pathlib.Path, an object with 'read' and 'seek' methods, or a length-1 dict of {file_path: object_path}, not None
File doesn't exit ../approval/higgsCombineGluGluToSUEP_HT1000_T2p00_mS125.000_mPhi8.000_T2.000_modehadronic_TuneCP5_13TeV-pythia8.HybridNew.mH125.root
'path' must be a string, pathlib.Path, an object with 'read' and 'seek' methods, or a length-1 dict of {file_path: object_path}, not None
File doesn't exit ../approval/higgsCombineGluGluToSUEP_HT1000_T8p00_mS125.000_mPhi2.000_T8.000_modegeneric_TuneCP5_13TeV-pythia8.HybridNew.mH125.root
'path' must be a string, pathlib.Path, an object with 'read' and 'seek' methods, or a length-1 dict of {file_path: object_path}, not N

In [16]:
decays = ['generic', 'leptonic', 'hadronic']

# divide the table by decay mode, and save each separetely
for decay in decays:
    
    good_rows = asymptotic_table[:,3]==decay  # only select one decay mode
    good_rows[0] = True          # also save the labels
    table_decay = asymptotic_table[good_rows, :] 
    table_decay = table_decay[:, np.arange(asymptotic_table.shape[1]) != 3] # exclude the decay column from output table

    fname = ofile.format('AsymptoticLimits', decay)
    with open(fname, "w") as file:
        file.write(tabulate(table_decay[1:], headers=table_decay[0,:]))

    print("Written", fname)

Written ../approval/hep_AsymptoticLimits_generic.txt
Written ../approval/hep_AsymptoticLimits_leptonic.txt
Written ../approval/hep_AsymptoticLimits_hadronic.txt


In [17]:
decays = ['generic', 'leptonic', 'hadronic']

# divide the table by decay mode, and save each separetely
for decay in decays:
    
    good_rows = toys_table[:,3]==decay  # only select one decay mode
    good_rows[0] = True          # also save the labels
    table_decay = toys_table[good_rows, :] 
    table_decay = table_decay[:, np.arange(toys_table.shape[1]) != 3] # exclude the decay column from output table

    fname = ofile.format('HybridNew', decay)
    with open(fname, "w") as file:
        file.write(tabulate(table_decay[1:], headers=table_decay[0,:]))

    print("Written", fname)

Written ../approval/hep_HybridNew_generic.txt
Written ../approval/hep_HybridNew_leptonic.txt
Written ../approval/hep_HybridNew_hadronic.txt
